## Example on using the `imnet` CDR3 network analysis library

In [1]:
import os
os.environ['SPARK_HOME'] = os.path.join(os.path.expanduser('~'),'spark')

import subprocess, re
from IPython.display import HTML

import sparkhpc
from sparkhpc.sparkjob import LSFSparkJob

import findspark
findspark.init()
import pyspark

import numpy as np
from scipy.sparse import csr_matrix 

import imnet

### Initialize a Spark cluster

Depending on your system setup, this will vary. If you are running on an HPC cluster, we recommend using [`sparkhpc`](https://github.com/rokroskar/sparkhpc) to manage your spark clusters in the batch queue. 

In [2]:
sj = LSFSparkJob(ncores=10)

In [3]:
sj.wait_to_start()

In [4]:
sc = sparkhpc.start_spark(master=sj.master_url())

Check that Spark is working:

In [6]:
sc.parallelize(range(sc.defaultParallelism)).collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

### Generating a random string sample

In [8]:
strings = imnet.random_strings.generate_random_sequences(5000)

In [9]:
strings[:10]

['TSGLKQ',
 'CATKSAKRFWWAAMWI',
 'KAFKLFF',
 'LGATNPE',
 'VDHRRWKTQHNVTMNAFSK',
 'MRQIKM',
 'MILAARYNIEMYSMVF',
 'YNLGMYASTVRT',
 'NKCVNEMLIHMP',
 'LAFENVSSTGTQC']

### Creating the distance matrix 
Now we create the weighted adjacency matrix, i.e. the distance matrix. If $S_i$ is the ith string, then the $i,j$ component of the matrix is the Levenshtein distance between strings $S_i$ and $S_j$, i.e. $D_{i,j}=L_d(S_i,S_j)$. We only keep the weights with $w_i = L_d(S_i) \le L_{d,max}$.

In [11]:
%%time 
mat_arr = np.array(list(imnet.process_strings.distance_matrix(strings, min_ld=1, max_ld=2)))
mat = csr_matrix((mat_arr[:,2], (mat_arr[:,0], mat_arr[:,1])), (len(strings), len(strings)))

INFO:imnet.process_strings:number of strings 5000


CPU times: user 4.93 s, sys: 0 ns, total: 4.93 s
Wall time: 4.93 s


In [12]:
mat

<5000x5000 sparse matrix of type '<type 'numpy.int64'>'
	with 988 stored elements in Compressed Sparse Row format>

In this case, the calculations were done only on the local machine. If we pass a `SparkContext` to the `sc` keyword, the work is distributed across the Spark cluster. 

In [15]:
%%time 
mat_spark_arr = np.array(imnet.process_strings.distance_matrix(strings, min_ld=1, max_ld=2, sc=sc).collect())
mat_spark = csr_matrix((mat_spark_arr[:,2], (mat_spark_arr[:,0], mat_spark_arr[:,1])), (len(strings), len(strings)))

INFO:imnet.process_strings:number of strings 5000


CPU times: user 57 ms, sys: 7 ms, total: 64 ms
Wall time: 1.48 s


In [16]:
mat_spark

<5000x5000 sparse matrix of type '<type 'numpy.int64'>'
	with 988 stored elements in Compressed Sparse Row format>

In [17]:
print 'The two matrices are identical: %s'%(not np.any((mat!=mat_spark).todense()))

The two matrices are identical: True


Finally, the distance matrix can be turned into a weighted graph. The code below constructs the sparse distance matrix like we have done above and uses it to make a graph. 

### Generate a graph from the distance matrix

In [18]:
%time g = imnet.process_strings.generate_graph(strings, sc=sc, min_ld=1, max_ld=2)

INFO:imnet.process_strings:number of strings 5000


CPU times: user 65 ms, sys: 4 ms, total: 69 ms
Wall time: 1.82 s


In [19]:
print 'Number of edges: %d\nNumber of nodes: %d'%(g.number_of_edges(), g.number_of_nodes())

Number of edges: 988
Number of nodes: 5000


In [20]:
g.nodes()[:10]

['SYQLSDL',
 'CTYELIDRDMVIPTVKW',
 'YQKG',
 'QPLELNVVPDS',
 'NGEGGKSEV',
 'VKGSYIFDCV',
 'AQEDKVGCTQQCSFD',
 'LDYTH',
 'GFQDACRTFDP',
 'YASFR']

### Generate degrees

One of the key things we want to know is the degree distribution of the sequences. The usual graph methods only give us the overall degrees independent of edge weight, or in our case, edit distance. We have implemented a function `produce_degrees` that counts the degree for each node per $L_d$:

In [22]:
%time degrees = imnet.process_strings.generate_degrees(strings, sc=sc, min_ld=1, max_ld=2)

INFO:imnet.process_strings:number of strings 5000


CPU times: user 24 ms, sys: 13 ms, total: 37 ms
Wall time: 4.05 s


This is a 2D $(N \times M)$ array, where $N$ is the number of strings and $M$ is the number of levels, i.e. if our maximum Levenshtein distance is 2, we have two levels. 

In [23]:
degrees

array([[0, 0, 1],
       [0, 0, 0],
       [0, 0, 0],
       ..., 
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int32)

The layout is such that the rows of the array correspond to the strings in our string map, e.g. lets say we want to know the degrees for string `YQKG`, we can do

In [25]:
string_map = {s:i for i,s in enumerate(strings)}

In [28]:
string_id = string_map['YQKG']
degrees[string_id]

array([0, 0, 4], dtype=int32)

This means that this node has four $L_d=2$ connections and zero $L_d=1$ connections

In [29]:
g_degrees = g.degree()
degree_match = np.all([g_degrees[s]==d.sum() for s,d in zip(strings,degrees)])
print 'Degrees from networkx match our calculation: %s'%degree_match

Degrees from networkx match our calculation: True


### Generate a distributed graph

For very large samples or very deep graphs (high $L_d$) it might be prohibitive to generate graphs locally -- instead we could use the distributed graph library [`GraphFrames`](http://graphframes.github.io/index.html) that can be used with `Spark`. 

In [34]:
g_rdd = imnet.process_strings.generate_spark_graph(strings, sc, max_ld=2).cache()

INFO:imnet.process_strings:number of strings 5000


The IDs of the vertices correspond to our `string_map` constructed above:

In [35]:
g_rdd.vertices.show()

+---+-------------------+
| id|             string|
+---+-------------------+
|  0|             TSGLKQ|
|  1|   CATKSAKRFWWAAMWI|
|  2|            KAFKLFF|
|  3|            LGATNPE|
|  4|VDHRRWKTQHNVTMNAFSK|
|  5|             MRQIKM|
|  6|   MILAARYNIEMYSMVF|
|  7|       YNLGMYASTVRT|
|  8|       NKCVNEMLIHMP|
|  9|      LAFENVSSTGTQC|
| 10|               ASFD|
| 11|      HCEMEGRLHLAIK|
| 12|            DVMFPTA|
| 13|               DRER|
| 14|     MFYSMFPINHPCGY|
| 15| HVHWRKGAVATYNEVYRM|
| 16|         HFQKEDQGLH|
| 17|  VAFHYSHFQTWHVQLYF|
| 18|    CEWFSPTIHGVPYVD|
| 19|       DACVSTHPSINF|
+---+-------------------+
only showing top 20 rows



In [36]:
g_rdd.edges.show()

+----+----+------+
| src| dst|weight|
+----+----+------+
|1858|1030|     2|
|4161| 447|     2|
|4161|1442|     2|
|4161|2841|     2|
|2314|1875|     2|
|2111|1839|     2|
|4481| 705|     2|
|4481|1118|     2|
|4481|1413|     2|
|4481|1899|     2|
|4481|1936|     2|
|4481|2884|     2|
|4481|3062|     2|
|4308|1476|     2|
|4308|2555|     2|
|4308|3595|     2|
|4781| 190|     1|
|4781|1678|     2|
|4781|1946|     2|
|4781|2928|     2|
+----+----+------+
only showing top 20 rows



There are several graph algorithms we can try:

In [37]:
%time comp_rdd = g_rdd.connectedComponents()

CPU times: user 69 ms, sys: 28 ms, total: 97 ms
Wall time: 34.2 s


In [38]:
comp_rdd.sort('id').show()

+---+-------------------+---------+
| id|             string|component|
+---+-------------------+---------+
|  0|             TSGLKQ|        0|
|  1|   CATKSAKRFWWAAMWI|        1|
|  2|            KAFKLFF|        2|
|  3|            LGATNPE|        3|
|  4|VDHRRWKTQHNVTMNAFSK|        4|
|  5|             MRQIKM|        5|
|  6|   MILAARYNIEMYSMVF|        6|
|  7|       YNLGMYASTVRT|        7|
|  8|       NKCVNEMLIHMP|        8|
|  9|      LAFENVSSTGTQC|        9|
| 10|               ASFD|       10|
| 11|      HCEMEGRLHLAIK|       11|
| 12|            DVMFPTA|       12|
| 13|               DRER|       10|
| 14|     MFYSMFPINHPCGY|       14|
| 15| HVHWRKGAVATYNEVYRM|       15|
| 16|         HFQKEDQGLH|       16|
| 17|  VAFHYSHFQTWHVQLYF|       17|
| 18|    CEWFSPTIHGVPYVD|       18|
| 19|       DACVSTHPSINF|       19|
+---+-------------------+---------+
only showing top 20 rows



Top components:

In [39]:
gb = comp_rdd.groupBy('component')
gb.count().sort('count', ascending=False).show()

+---------+-----+
|component|count|
+---------+-----+
|       10|  487|
|     1910|    3|
|      641|    3|
|      358|    2|
|     2004|    2|
|     2713|    2|
|     2334|    2|
|     2449|    2|
|      269|    2|
|     1848|    2|
|        0|    2|
|     1334|    2|
|      871|    2|
|      951|    2|
|     2734|    2|
|      536|    2|
|     1646|    2|
|      188|    2|
|     1821|    2|
|     1514|    2|
+---------+-----+
only showing top 20 rows



In [36]:
sc.stop()